In [2]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0],
                                '..', 'module'))
import wiki
import pickle
import numpy as np
import gensim.models as gm
import gensim.utils as gu
import gensim.corpora as gc
import gensim.matutils as gmat
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as smp

In [4]:
path_base = '/Users/harangju/Developer/data/wiki/partition/'
name_xml = 'enwiki-20190720-pages-articles-multistream1.xml-p10p30302.bz2'
name_index = 'enwiki-20190720-pages-articles-multistream-index1.txt-p10p30302.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)
corpus = wiki.Corpus(dump)

Dump: Loading index...
Dump: Loaded.


In [ ]:
dct = gc.Dictionary(corpus)
tfidf = gm.TfidfModel(dictionary=dct)

In [ ]:
tfidf.save('/Users/harangju/Developer/data/wiki/models/tfidf-small.model')

In [ ]:
corpus[1][:5]

In [ ]:
tfidf[dct.doc2bow(corpus[1])][:5]

In [ ]:
vecs = [tfidf[dct.doc2bow(corpus[i])] for i in range(10)]
svecs = gmat.corpus2csc(vecs)
svecs

In [ ]:
norm = lambda v1,v2: v1.transpose()*v2 \
       / np.sqrt(v1.power(2).sum()) \
       / np.sqrt(v2.power(2).sum())
print(norm(svecs[:,1],svecs[:,3]))

### Test big model
See `preparation/build-tfidf-model.ipynb`.

In [6]:
path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [7]:
corpus[1][:5]

['transhumanism', 'abbreviated', 'as', 'or', 'is']

In [28]:
tfidf[dct.doc2bow(corpus[1])][:5]

### Weight graph

In [ ]:
topic = 'earth science'
network = wiki.Net()
network.load_graph('/Users/harangju/Developer/data/wiki/graphs/'+topic+'.gexf')

In [ ]:
def set_weights(graph, dump, model, dct):
    for n1, n2 in graph.edges:
        p1 = gu.simple_preprocess(dump.load_page(n1).strip_code())
        p2 = gu.simple_preprocess(dump.load_page(n2).strip_code())
        graph[n1][n2]['weight'] = smp.cosine_similarity(X=model[dct.doc2bow(p1)],
                                                        Y=model[dct.doc2bow(p2)])

In [ ]:
network.graph['Biosphere']

In [ ]:
print(network.graph['Biosphere']['Earth'])
print(network.graph['Biosphere']['Earth'])

In [ ]:
set_weights(network.graph, dump, model, dct)

In [ ]:
print(network.graph['Biosphere']['Earth'])
print(network.graph['Biosphere']['Earth'])